In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam

# -------------------- Settings --------------------
data_dir = "../data"
file_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
features = ['MaxTemp', 'MinTemp', 'StdCellV', 'Current', 'PackV']
timesteps = 30

# -------------------- Load & Preprocess --------------------
df_all = []
for file in file_list:
    df = pd.read_csv(os.path.join(data_dir, file), encoding='cp949')
    df_all.append(df[features])
data = pd.concat(df_all).reset_index(drop=True)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Sliding window to make sequences
sequences = []
for i in range(len(data_scaled) - timesteps):
    sequences.append(data_scaled[i:i+timesteps])
sequences = np.array(sequences)

# -------------------- Model --------------------
inputs = Input(shape=(timesteps, len(features)))
encoded = LSTM(64, return_sequences=False)(inputs)
bottleneck = Dense(3, activation='sigmoid')(encoded)  # RGB

x = RepeatVector(timesteps)(bottleneck)
x = LSTM(64, return_sequences=True)(x)
decoded = TimeDistributed(Dense(len(features)))(x)

model = Model(inputs, decoded)
model.compile(optimizer=Adam(1e-3), loss='mse')
model.fit(sequences, sequences, epochs=10, batch_size=128, verbose=1)

# -------------------- RGB Visualization --------------------
encoder = Model(inputs, bottleneck)
rgb_encoded = encoder.predict(sequences)

plt.figure(figsize=(14, 2))
for i, rgb in enumerate(rgb_encoded):
    plt.plot(i, 1, 'o', color=rgb)
plt.title("Battery Health Visualization via RGB")
plt.yticks([])
plt.xlabel("Time")
plt.tight_layout()

os.makedirs("../images", exist_ok=True)
plt.savefig("../images/visualization_result.png")
plt.show()
